# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import folium

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,clarksdale,34.2001,-90.5709,6.62,84,0,2.06,US,1708417012
1,1,adamstown,-25.0660,-130.1015,25.52,87,45,5.27,PN,1708417013
2,2,quelimane,-17.8786,36.8883,26.97,94,75,3.93,MZ,1708417013
3,3,korla,41.7597,86.1469,-2.78,24,100,2.48,CN,1708417014
4,4,port elizabeth,-33.9180,25.5701,31.17,75,0,4.63,ZA,1708417015


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
map = folium.Map(location=[city_data_df["Lat"].mean(), city_data_df["Lng"].mean()], zoom_start=2)

# Add points to the map
for _, city in city_data_df.iterrows():
    folium.CircleMarker(
        location=[city["Lat"], city["Lng"]],
        radius=city["Humidity"] / 20,  # Adjust size to be reasonable on the map
        popup=f"{city['City']}, {city['Country']}: Humidity {city['Humidity']}%",
        color='blue',
        fill=True,
        fill_color='blue',
    ).add_to(map)

# Display the map
map

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
reduced_city_df = city_data_df.loc[(city_data_df['Wind Speed'] < 4) & 
                 (city_data_df['Cloudiness'] < 15) &
                 (city_data_df['Max Temp'] > 15) &
                 (city_data_df['Max Temp'] < 30) ]


reduced_city_df.dropna()

# Display sample data
reduced_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,54,marat,25.0701,45.4615,21.70,27,0,0.61,SA,1708417052
106,106,ghat,24.9647,10.1728,16.82,23,0,1.94,LY,1708417086
113,113,afif,23.9065,42.9172,19.67,22,4,1.27,SA,1708417091
137,137,dutse,11.7594,9.3392,26.09,13,6,3.14,NG,1708417106
148,148,acapulco de juarez,16.8634,-99.8901,22.90,78,0,2.06,MX,1708417113


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = reduced_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
54,marat,SA,25.0701,45.4615,27,
106,ghat,LY,24.9647,10.1728,23,
113,afif,SA,23.9065,42.9172,22,
137,dutse,NG,11.7594,9.3392,13,
148,acapulco de juarez,MX,16.8634,-99.8901,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey': geoapify_key,
    'categories': "accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
marat - nearest hotel: No hotel found
ghat - nearest hotel: فندق تاسيلي
afif - nearest hotel: Samaya Royal Afif
dutse - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: Hotel del Valle
coyhaique - nearest hotel: No hotel found
celestun - nearest hotel: Hotel Flamingo Playa
las toscas - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
puerto madero - nearest hotel: No hotel found
quarai - nearest hotel: Hotel União
cozumel - nearest hotel: Hotel Isla Esmeralda
yallahs - nearest hotel: No hotel found
agadez - nearest hotel: No hotel found
concordia - nearest hotel: D'Charruas
kandiaro - nearest hotel: No hotel found
ambovombe - nearest hotel: La source
necochea - nearest hotel: Cabañas Río Quequén
perugorria - nearest hotel: No hotel found
thames - nearest hotel: Kopu Station Hotel
chui - nearest hotel: Alerces
les anglais - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
champerico - nearest hotel: Hote

,City,Country,Lat,Lng,Humidity,Hotel Name
54,marat,SA,25.0701,45.4615,27,No hotel found
106,ghat,LY,24.9647,10.1728,23,فندق تاسيلي
113,afif,SA,23.9065,42.9172,22,Samaya Royal Afif
137,dutse,NG,11.7594,9.3392,13,No hotel found
148,acapulco de juarez,MX,16.8634,-99.8901,78,Hotel del Valle


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
# Configure the map plot
map_hotels = folium.Map(location=[hotel_df["Lat"].mean(), hotel_df["Lng"].mean()], zoom_start=2)

# Add markers for each city to the map
for index, row in hotel_df.iterrows():
    # Define the message to display on hover
    hover_message = f"City: {row['City']}, Country: {row['Country']}\nHotel: {row['Hotel Name']}"
    
    # Create and add a marker to the map
    folium.Marker(
        [row["Lat"], row["Lng"]],
        popup=hover_message,
        tooltip=hover_message  
    ).add_to(map_hotels)

# Display the map
map_hotels